In [1]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2
import numpy as np
import datetime
from sktime_neuro.utils.pickle_wrapper import deserialise_if_exists, serialise_if_doesnt_exist
from sktime_neuro.datasets.matching_pennies_util import *
from sktime_neuro.transformations.multivariate_detrender import ColumnDetrender
from sktime_neuro.transformations.PREP import do_prep, build_prep_params
from sktime_neuro.transformations.series_to_panel.eeg_epoching import epoch
from sktime_neuro.utils.mne_processing import create_annotation
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import *

In [2]:
def detrend(data, n_jobs=1):
    dt = ColumnDetrender(data)
    return dt.detrend(n_jobs=n_jobs)

In [3]:
def process(sid):
    raw = get_raw(sid)
    sf = raw.info['sfreq']

    annot = create_annotation(raw)
    data = raw.get_data().transpose()
    detrended = detrend(data, -1)
    return (annot, detrended, sf)

def leaveOneOut(leaveOut: int):
    leftout = None
    datasets = []
    for i in range(5,12):
        if i == leaveOut:
            leftout = process(i)
        else:
            datasets.append(process(i))
    return datasets, leftout

In [4]:
datasets, leftout = leaveOneOut(5)

Reading 0 ... 9348356  =      0.000 ...  1869.671 secs...


/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/datasets/matching_pennies_util.py:10: RuntimeWarning: Did not find any eeg.json associated with sub-05_task-matchingpennies.

The search_str was "/home/patchouli/programming/foss/sktime-neuro/cache/eeg_matchingpennies/sub-05/**/eeg/sub-05*eeg.json"
  return read_raw_bids(bids_path=bidspath, verbose=False)


Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 16501 samples (3.300 sec)

Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 16501 samples (3.300 sec)



KeyboardInterrupt: 

In [ ]:
onsetlabels = ["raised-right/match-true", "raised-right/match-false",
               "raised-left/match-true", "raised-left/match-false"]

def epoch_dataset(dataset):
    epochs, labels = epoch(dataset[1], dataset[0], onsetlabels, [-0.5, 0.5], sfreq=dataset[2])
    return (epochs, labels)

def epoch_all(datasets):
    epoched = []
    for dataset in datasets:
        epoched.append(epoch_dataset(dataset))

    return epoched

In [ ]:
epoched = epoch_all(datasets)
l1a_epochs, l1a_labels = epoch_dataset(leftout)

In [ ]:
rocket = RocketClassifier(n_jobs=-1)
metrics = np.empty([len(datasets),3])
i = 0
for trainset, trainlabs in epoched:
    rocket.fit(trainset, trainlabs)
    ypred = rocket.predict(l1a_epochs)
    #print(classification_report(ypred, l1a_labels))
    acc = accuracy_score(ypred, l1a_labels)
    f1 = f1_score(ypred, l1a_labels, average='weighted')
    precision = precision_score(ypred, l1a_labels, average='weighted')
    x = np.array([acc, f1, precision])
    metrics[i] = x
    i += 1